# Create MXNet Container

In [ ]:
# Create the binary
!git clone https://github.com/aws/sagemaker-mxnet-container.git
!cd sagemaker-mxnet-container
!git checkout v3.1.0
!python setup.py sdist

In [ ]:
# MXNet 1.4.1, Python 3, CPU
!cp dist/sagemaker_mxnet_container-*.tar.gz docker/1.4.1/py3/sagemaker_mxnet_container.tar.gz
!cd docker/1.4.1/py3/
!docker build -t preprod-mxnet:1.4.1-cpu-py3 -f Dockerfile.cpu .

In [ ]:
!cd ../../../
!pip install -e .[test]

In [ ]:
!tox test/unit

In [ ]:
# Create an IAM role called [SageMakerRole] with AmazonSageMakerFullAccess manually

In [ ]:
!tox -- test/integration/local --docker-base-name preprod-mxnet \
                              --tag 1.4.1-cpu-py3 \
                              --py-version 3 \
                              --framework-version 1.4.1 \
                              --processor cpu

In [ ]:
# upload container to ECS
# TODO

In [ ]:
!tox -- test/integration/sagemaker --aws-id 12345678910 \
                                  --docker-base-name preprod-mxnet \
                                  --instance-type ml.m4.xlarge \
                                  --tag 1.4.1-cpu-py3